In [2]:
import os

import pandas as pd

DATA_DIR = 'data/staging/logs/'

In [7]:
df = pd.read_csv(DATA_DIR + 'vmgr_sessions.csv', names=['date','auth'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6831 entries, 0 to 6830
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    6831 non-null   object
 1   auth    6831 non-null   object
dtypes: object(2)
memory usage: 106.9+ KB


In [8]:
df['date'] = pd.to_datetime(df['date'])
df.head()

,date,auth
0,2018-12-17 20:14:05,jmancino
1,2018-12-17 20:34:33,jmancino
2,2018-12-18 07:30:14,gkeitner
3,2018-12-18 07:36:23,rsoemedi
4,2018-12-18 07:37:46,dmurphyd


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6831 entries, 0 to 6830
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    6831 non-null   datetime64[ns]
 1   auth    6831 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 106.9+ KB
